In [1]:
library(edgeR)
library(magrittr)
library(ggplot2)
library(DESeq2)
library(tidyverse)

Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

## Automatise

In [2]:
path_csvs = '/home/jovyan/projects/kk14_DCM-lymphoid/results/for_edgeR/210526/'
path_csvs

[1] "/home/jovyan/projects/kk14_DCM-lymphoid/results/for_edgeR/210526/"

In [3]:
x=read.csv(paste0(path_csvs, "ALL_CELLSTATE_PSEUDOBULK_RV.csv"), row.names=1, check.names=FALSE)
genes_tofilter=read.csv(paste0(path_csvs, "ALL_CELLSTATE_PSEUDOBULK_FILTERING_RV.csv"), check.names=FALSE)
colnames(genes_tofilter) <- gsub("mutation.negative", 'PVneg', colnames(genes_tofilter))
colnames(genes_tofilter)[1]='X' # since check.names=FALSE remove 'X' from the first column name

# Only needed for the column cell_state
CELLTYPE_STATE <- read.csv(paste0(path_csvs, "CELLSTATE_TRANSLATION_TABLE_RV.csv"))
colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv(paste0(path_csvs, "ABSOLUTE_CELLSTATES_NUMBER_RV.csv"))

l <- strsplit(colnames(x), "__")

In [4]:
head(genes_tofilter)

,X,control_PC1,control_vCM1.1,control_SMC1.1,control_vCM1.0,control_vCM3.0,control_vCM2,control_vCM4,control_Mast,control_vFB1.0,⋯,PKP2_CD8T_te_IFNGhi,PKP2_AD1.0,PKP2_cDC1,PKP2_MY16,PKP2_Meso,PKP2_MY10,PKP2_prolif_Lymphoids,PKP2_unclassified.2,PKP2_NC1.3,PKP2_NC6
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,MIR1302-2HG,0.000000000,0.00000000,0.00000000,2.769699e-05,0.00000000,0.000000000,0.00000000,0.000000000,0.00000000,⋯,0.000000000,0,0,0,0,0.00000000,0,0,0.00000000,0
2,FAM138A,0.000000000,0.00000000,0.00000000,0.000000e+00,0.00000000,0.000000000,0.00000000,0.000000000,0.00000000,⋯,0.000000000,0,0,0,0,0.00000000,0,0,0.00000000,0
3,OR4F5,0.000000000,0.00000000,0.00000000,0.000000e+00,0.00000000,0.000000000,0.00000000,0.000000000,0.00000000,⋯,0.000000000,0,0,0,0,0.00000000,0,0,0.00000000,0
4,AL627309.1,0.004965798,0.01577006,0.00313071,1.578136e-02,0.02219098,0.025318187,0.02091467,0.007936508,0.00721482,⋯,0.001824818,0,0,0,0,0.02305987,0,0,0.04444445,0
5,AL627309.3,0.000000000,0.00000000,0.00000000,0.000000e+00,0.00000000,0.000000000,0.00000000,0.000000000,0.00000000,⋯,0.000000000,0,0,0,0,0.00000000,0,0,0.00000000,0
6,AL627309.2,0.000000000,0.00000000,0.00000000,0.000000e+00,0.00000000,0.001874706,0.00000000,0.000000000,0.00000000,⋯,0.000000000,0,0,0,0,0.00000000,0,0,0.00000000,0


In [5]:
CELLTYPE_STATE %>% filter(cell_type=='Lymphoid')

cell_states,cell_type
<chr>,<chr>
NK_CD56hi,Lymphoid
CD8T_cytox,Lymphoid
CD4T_naive,Lymphoid
CD8T_trans,Lymphoid
CD4T_act,Lymphoid
CD8T_em,Lymphoid
NK_CD16hi,Lymphoid
MAIT-like,Lymphoid
NK_CD16hiIFNGhi,Lymphoid


In [6]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Genotype", "cell_state", 'Patient', 'X10X_version', 'Gender')

head(meta.data)

,Genotype,cell_state,Patient,X10X_version,Gender
,<chr>,<chr>,<chr>,<chr>,<chr>
LMNA__AD1.0__H06__V3__f,LMNA,AD1.0,H06,V3,f
LMNA__AD1.0__H24__V3__f,LMNA,AD1.0,H24,V3,f
LMNA__AD1.0__H28__V3__m,LMNA,AD1.0,H28,V3,m
LMNA__AD1.1__H06__V3__f,LMNA,AD1.1,H06,V3,f
LMNA__AD1.1__H24__V3__f,LMNA,AD1.1,H24,V3,f
LMNA__AD1.1__H25__V3__m,LMNA,AD1.1,H25,V3,m


In [7]:
GENOTYPES <- unique(meta.data$Genotype)[-6]
GENOTYPES

[1] "LMNA"  "PKP2"  "PVneg" "RBM20" "TTN"

In [8]:
CELL_LEVEL <- "CELLSTATE"
REGION <- "RV"

In [9]:
CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
CELLTYPE_FILTER_SUB[,2:ncol(CELLTYPE_FILTER_SUB)] <- apply(CELLTYPE_FILTER_SUB[,-1], 2, function(i) i>5)

In [10]:
as.character(GENOTYPES)

[1] "LMNA"  "PKP2"  "PVneg" "RBM20" "TTN"

In [11]:
as.character(unique(meta.data$cell_state))

[1] "AD1.0"            "AD1.1"            "AD2"              "B"               
 [5] "CD16+ Mo"         "CD4T_act"         "CD4T_naive"       "CD4T_reg"        
 [9] "CD8T_cytox"       "CD8T_em"          "CD8T_te_IFNGhi"   "CD8T_te"         
[13] "CD8T_trans"       "EC1.0"            "EC2.0"            "EC5.0"           
[17] "EC6.0"            "EC7.0"            "EC8.0"            "ILC"             
[21] "MAIT-like"        "MY10"             "MY12"             "MY14"            
[25] "MY15"             "MY16"             "MY1"              "MY2"             
[29] "MY3"              "MY4"              "MY5"              "MY7"             
[33] "MY8"              "MY9"              "Mast"             "Meso"            
[37] "NC1.0"            "NC1.1"            "NC1.2"            "NC1.3"           
[41] "NC1.4"            "NC1.5"            "NC2"              "NC3"             
[45] "NC4"              "NC5"              "NK_CD16hiIFNGhi"  "NK_CD16hi"       
[49] "NK_CD56hi"        "PC1"              "PC2"              "PC3"             
[53] "Plasma"           "SMC1.1"           "SMC1.2"           "SMC2"            
[57] "cDC1"             "cDC2"             "prolif Myeloids"  "prolif_Lymphoids"
[61] "unclassified.1"   "vCM1.0"           "vCM1.1"           "vCM1.2"          
[65] "vCM1.3"           "vCM2"             "vCM3.0"           "vCM3.1"          
[69] "vCM4"             "vCM5"             "vFB1.0"           "vFB1.1"          
[73] "vFB1.2"           "vFB2"             "vFB3"             "vFB4"            
[77] "NC6"              "unclassified.2"   "AD3"

In [12]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    for(CELL_STATE in as.character(unique(meta.data$cell_state))){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_states==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data$Genotype %in% c("control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data$Genotype %in% c("control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_state %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_state %in% c(CELL_STATE)),]
        
        # Suggestion: Remove samples with less than 5 nuclei 
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_states==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        
        # The test is only run, if 2 conditions have at least 2 patients
        if (length(unique(meta.data_sub$Genotype))==2 &
           all(table(meta.data_sub$Genotype)>2)
           ){
             # Prepare DGEList object
            meta.data_sub$Genotype <- as.factor(as.character(meta.data_sub$Genotype))
            dge <- DGEList(counts=x_sub, group=meta.data_sub$Genotype)
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
                    genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            dge <- calcNormFactors(dge)
            cdr <- scale(colMeans(x_sub > 0))
            design <- model.matrix(~ cdr + meta.data_sub$Genotype)
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        }
    }
}


###START: LMNA #####



###FINISHED: AD2 #####



###FINISHED: CD4T_act #####



###FINISHED: CD4T_naive #####



###FINISHED: CD8T_em #####



###FINISHED: CD8T_te #####



###FINISHED: CD8T_trans #####



###FINISHED: EC1.0 #####



###FINISHED: EC2.0 #####



###FINISHED: EC5.0 #####



###FINISHED: EC6.0 #####



###FINISHED: EC7.0 #####



###FINISHED: EC8.0 #####



###FINISHED: MY10 #####



###FINISHED: MY12 #####



###FINISHED: MY1 #####



###FINISHED: MY2 #####



###FINISHED: MY3 #####



###FINISHED: MY4 #####



###FINISHED: MY5 #####



###FINISHED: MY7 #####



###FINISHED: MY8 #####



###FINISHED: MY9 #####



###FINISHED: NC1.0 #####



###FINISHED: NC1.4 #####



###FINISHED: NC2 #####



###FINISHED: NK_CD16hi #####



###FINISHED: NK_CD56hi #####



###FINISHED: PC1 #####



###FINISHED: PC2 #####



###FINISHED: PC3 #####



###FINISHED: SMC1.1 #####



###FINISHED: SMC1.2 #####



###FINISHED: SMC2 #####



###FINISHED: cDC2 #####



###FINISHED: vCM1.0 #####


In [13]:
meta.data_sub

Genotype,cell_state,Patient,X10X_version,Gender
<chr>,<chr>,<chr>,<chr>,<chr>


In [14]:
head(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A1BG,1.2205588,4.281900,0.8834923,0.3472479,0.8136900,F,0.089541014,0.010675382,0.03928571,RV,CELLSTATE,AD2,AD,control_LMNA
2,A1BG-AS1,1.7543292,4.103563,1.6803753,0.1948759,0.6907911,F,0.160653260,0.003703704,0.01916667,RV,CELLSTATE,AD2,AD,control_LMNA
3,A1CF,-1.6756031,3.987134,1.1340045,0.3021312,0.7803930,F,0.107686661,0.018518519,0.00000000,RV,CELLSTATE,AD2,AD,control_LMNA
4,A2M,-0.1773584,6.034037,0.1038335,0.7472772,0.9829653,F,0.007461817,0.265638230,0.23559526,RV,CELLSTATE,AD2,AD,control_LMNA
5,A2M-AS1,-0.5136542,4.170491,0.1565951,0.6923112,0.9701587,F,0.013157220,0.018518519,0.01428571,RV,CELLSTATE,AD2,AD,control_LMNA
6,A2ML1,-2.0343873,4.098135,1.9439397,0.1632427,0.6770706,F,0.169366072,0.012929809,0.20000000,RV,CELLSTATE,AD2,AD,control_LMNA


In [15]:
tail(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
7613121,ZXDB,0.2745719,3.865191,0.1260335,0.750507607,1.00000000,NA,NA,0.0023579134,0.006797264,RV,CELLSTATE,vFB4,FB,control_TTN
7613122,ZXDC,0.2828352,6.474848,0.8021216,0.372774272,0.69635381,F,0.1571700,0.1357661000,0.231421230,RV,CELLSTATE,vFB4,FB,control_TTN
7613123,ZYG11A,0.9231232,3.793061,1.4493956,0.394627470,1.00000000,NA,NA,0.0001539646,0.005780245,RV,CELLSTATE,vFB4,FB,control_TTN
7613124,ZYG11B,0.2472182,6.315714,0.5491753,0.460523752,0.76107777,F,0.1185710,0.2334743100,0.215667720,RV,CELLSTATE,vFB4,FB,control_TTN
7613125,ZYX,-1.9032982,5.071323,11.9903377,0.000810309,0.01923671,F,1.7158692,0.0724367950,0.022348622,RV,CELLSTATE,vFB4,FB,control_TTN
7613126,ZZEF1,0.3911716,6.774139,1.7548549,0.188516571,0.52174790,F,0.2825393,0.1853984100,0.319265460,RV,CELLSTATE,vFB4,FB,control_TTN


## Subset lymphoid DEGs and save

In [16]:
final_df %>% pull(cell_type) %>% unique()

[1] "AD"       "Lymphoid" "EC"       "Myeloid"  "N"        "Mural"    "CM"      
[8] "FB"       "Mast"

In [17]:
lymphoid_df = final_df %>% filter(cell_type=='Lymphoid')
lymphoid_df

Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
A1BG,9.263341e-01,6.948326,8.644692e-01,0.352491873,1.0000000,NA,NA,0.003999689,0.007722008,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A1BG-AS1,2.998464e-01,7.146198,7.973731e-02,0.999680417,1.0000000,F,0.000000000,0.028211400,0.034920637,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A1CF,-8.203380e-17,6.734739,0.000000e+00,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A2M,1.781676e+00,8.559676,6.821962e+00,0.009004861,0.6817757,F,0.166358463,0.073815815,0.379279300,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A2M-AS1,-3.865866e-01,7.441517,2.010009e-01,0.653914508,0.9958960,F,0.001785998,0.039856820,0.117857150,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A2ML1,-8.203380e-17,6.734739,0.000000e+00,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A2ML1-AS1,6.655713e-01,6.957613,3.456985e-01,0.556558756,1.0000000,NA,NA,0.003999689,0.011003861,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A2ML1-AS2,-8.203380e-17,6.734739,0.000000e+00,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
A3GALT2,-8.203380e-17,6.734739,0.000000e+00,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA


In [18]:
write.csv(lymphoid_df, file='/home/jovyan/projects/kk14_DCM-lymphoid/results/DEGs_df/210526/LYMPHOIDS_ALLGENOTYPES_EDGER_RV.csv')

In [2]:
lymphoid_df = read.csv('/home/jovyan/projects/kk14_DCM-lymphoid/results/DEGs_df/210526/LYMPHOIDS_ALLGENOTYPES_EDGER_RV.csv', header=TRUE, row.names=1)
head(lymphoid_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A1BG,9.263341e-01,6.948326,0.86446918,0.352491873,1.0000000,NA,NA,0.003999689,0.007722008,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
2,A1BG-AS1,2.998464e-01,7.146198,0.07973731,0.999680417,1.0000000,FALSE,0.000000000,0.028211400,0.034920637,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
3,A1CF,-8.203380e-17,6.734739,0.00000000,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
4,A2M,1.781676e+00,8.559676,6.82196200,0.009004861,0.6817757,FALSE,0.166358463,0.073815815,0.379279300,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
5,A2M-AS1,-3.865866e-01,7.441517,0.20100092,0.653914508,0.9958960,FALSE,0.001785998,0.039856820,0.117857150,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA
6,A2ML1,-8.203380e-17,6.734739,0.00000000,1.000000000,1.0000000,NA,NA,0.000000000,0.000000000,RV,CELLSTATE,CD4T_act,Lymphoid,control_LMNA


In [3]:
dim(lymphoid_df)

[1] 1039678      15

In [4]:
# Filter table to make list shorter
lymphoid_df_sel = lymphoid_df %>% filter(PValue<0.05)
dim(lymphoid_df_sel)

[1] 20571    15

In [5]:
write.csv(lymphoid_df_sel, file='/home/jovyan/projects/kk14_DCM-lymphoid/for_paper/Supplementary_tables/LYMPHOIDS_DEGs_RV.csv')